In [1]:
!pip install arch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import yfinance as yf
import arch

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
os.chdir('/content/drive/My Drive/thesis/3rd_presentation')
df = pd.read_csv('non_nan_4.csv')

In [5]:
df

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG,inflation_change,unemp_change,treasury_yield_change
0,2000-02-01,4.1,170.000,6.661000,0.496560,-1.522563,-12.828964,-13.228004,-16.339821,-11.121498,-21.701151,2.220031,3.838386,-11.226228,54.440789,1.000,-0.1,-0.141500
1,2000-03-01,4.0,171.000,6.519500,0.511258,9.413333,22.264136,-0.218329,-2.079067,5.804243,0.913712,8.390897,6.420237,14.101954,6.922258,1.000,-0.1,-0.141500
2,2000-04-01,3.8,170.900,6.256522,1.327803,-3.266805,5.567379,-8.205683,17.437698,23.153694,12.400712,-0.097663,2.559423,15.213674,7.370518,-0.100,-0.2,-0.262978
3,2000-05-01,4.0,171.200,5.990526,-0.181797,-1.572223,-0.148357,5.395746,8.484832,-1.296597,7.374072,20.863985,5.169310,5.638019,-8.163265,0.300,0.2,-0.265995
4,2000-06-01,4.0,172.200,6.440455,0.305565,1.728613,10.549735,5.788826,14.239888,31.641749,3.078671,2.813690,8.474599,8.076012,13.131313,1.000,0.0,0.449928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,-2.549213,2.541749,-0.110227,-13.817335,16.968326,1.546,-0.1,-0.274810
276,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,-1.089288,1.592445,-7.172811,-7.286115,5.451681,1.112,0.2,-0.084690
277,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,0.141189,12.873250,9.367574,0.566924,11.025813,0.160,-0.1,0.215342
278,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467,1.110,-0.1,-0.083799


In [6]:
def create_lags(data, tickers, covs, period):
    lagged_covs = []  # List to store the names of lagged columns

    for covariate in covs:
        for i in period:
            lagged_column_name = f'{covariate}_lag{i}'
            data[lagged_column_name] = data[covariate].shift(i)
            lagged_covs.append(lagged_column_name)

    lagged_tickers = []  # List to store the names of lagged columns

    for tick in tickers:
        for i in period:
            lagged_column_name = f'{tick}_lag{i}'
            data[lagged_column_name] = data[tick].shift(i)
            lagged_tickers.append(lagged_column_name)

    data.dropna(inplace=True)
    data = data.reset_index(drop=True)

    return data, lagged_covs, lagged_tickers

In [7]:
tickers = ['AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'NVS', 'PFE','ROG']
covs = ['unemployment_rate', 'CPI', 'treasury_yield', 'GDP_growth', 'SP500_return', 'inflation_change', 'unemp_change', 'treasury_yield_change']
period = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

data, lagged_covs, lagged_tickers = create_lags(df, tickers, covs, period)

<ipython-input-6-fe5811118561>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[lagged_column_name] = data[tick].shift(i)
<ipython-input-6-fe5811118561>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[lagged_column_name] = data[tick].shift(i)
<ipython-input-6-fe5811118561>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fra

In [8]:
df

,Date,unemployment_rate,CPI,treasury_yield,GDP_growth,SP500_return,AZN,BMY,JNJ,LLY,...,ROG_lag3,ROG_lag4,ROG_lag5,ROG_lag6,ROG_lag7,ROG_lag8,ROG_lag9,ROG_lag10,ROG_lag11,ROG_lag12
12,2001-02-01,4.2,176.000,5.160952,-0.195020,-9.538746,4.143180,2.851534,4.509829,0.837527,...,1.748252,13.043478,-9.318996,-2.447552,2.142857,13.131313,-8.163265,7.370518,6.922258,54.440789
13,2001-03-01,4.3,176.100,5.098947,-0.217255,-5.857225,5.615486,-6.323922,-9.828843,-3.186580,...,12.886598,1.748252,13.043478,-9.318996,-2.447552,2.142857,13.131313,-8.163265,7.370518,6.922258
14,2001-04-01,4.4,176.400,4.885455,0.756989,8.836250,-1.512969,-5.723911,10.300706,10.879156,...,-5.022831,12.886598,1.748252,13.043478,-9.318996,-2.447552,2.142857,13.131313,-8.163265,7.370518
15,2001-05-01,4.3,177.300,5.141000,0.461042,-0.560577,0.631283,-2.660517,0.487142,-0.352891,...,-8.846156,-5.022831,12.886598,1.748252,13.043478,-9.318996,-2.447552,2.142857,13.131313,-8.163265
16,2001-06-01,4.5,177.700,5.391364,-0.691442,-2.659796,-2.237527,-3.576694,3.433253,-12.344135,...,-0.112520,-8.846156,-5.022831,12.886598,1.748252,13.043478,-9.318996,-2.447552,2.142857,13.131313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-01-01,3.4,300.536,3.616190,0.390254,6.776820,-3.584079,0.972908,-7.489384,-5.928822,...,-2.707956,-3.448826,-6.956359,2.731887,-1.239735,-1.972516,-0.360696,-0.476186,0.018314,-0.018311
276,2023-02-01,3.6,301.648,3.531500,-0.442183,-2.514271,-0.290649,-4.328217,-6.217115,-9.568502,...,-53.665066,-2.707956,-3.448826,-6.956359,2.731887,-1.239735,-1.972516,-0.360696,-0.476186,0.018314
277,2023-03-01,3.5,301.808,3.746842,-0.442183,3.313488,8.035329,0.507544,1.862736,10.703390,...,9.446071,-53.665066,-2.707956,-3.448826,-6.956359,2.731887,-1.239735,-1.972516,-0.360696,-0.476186
278,2023-04-01,3.4,302.918,3.663043,-0.442183,1.985238,5.489119,-3.664707,5.612908,15.269915,...,16.968326,9.446071,-53.665066,-2.707956,-3.448826,-6.956359,2.731887,-1.239735,-1.972516,-0.360696


# 1 month ahead

18/06/2023

In [20]:
import pandas as pd
import numpy as np
import arch

def one_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha, t):
    VaR_forecasts = {}
    losses = {}

    for ticker in tickers:
        lagged_tickers_ticker = [tick for tick in lagged_tickers if tick.startswith(f'{ticker}_lag')]

        # Prepare the data
        train_data = data.loc[:t]
        test_data = data.loc[t:]

        # Specify the regressors and dependent variable
        X_train = train_data[lagged_covs + lagged_tickers_ticker]
        y_train = train_data[ticker]

        # Fit the GARCH(1, 1) model
        garch_model = arch.arch_model(y_train, x=X_train, vol='Garch', p=1, q=1)
        garch_model_result = garch_model.fit()

        # Obtain the predicted conditional volatility for t+h
        X_test = test_data[lagged_covs + lagged_tickers_ticker]
        conditional_volatility = garch_model_result.conditional_volatility
        VaR_forecast = conditional_volatility * np.percentile(garch_model_result.resid / conditional_volatility, alpha * 100)

        # Compute loss
        r = test_data[ticker]
        loss = tick_loss(alpha, r, VaR_forecast)

        VaR_forecasts[ticker] = VaR_forecast
        losses[ticker] = loss

    return VaR_forecasts, losses


In [11]:
alpha = 0.05
t = 255
VaR_forecasts, losses = one_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha, t)

# Access VaR forecasts and losses
for ticker in tickers:
    print(f'Ticker: {ticker}')
    print('VaR forecast:', VaR_forecasts[ticker])
    print('Loss:', losses[ticker])
    print('---')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1159.7636996500732
Iteration:      2,   Func. Count:     13,   Neg. LLF: 849.6317953982542
Iteration:      3,   Func. Count:     20,   Neg. LLF: 937.2452486616035
Iteration:      4,   Func. Count:     28,   Neg. LLF: 840.3751966360928
Iteration:      5,   Func. Count:     33,   Neg. LLF: 840.3670078951413
Iteration:      6,   Func. Count:     38,   Neg. LLF: 840.3446163506405
Iteration:      7,   Func. Count:     43,   Neg. LLF: 840.3430701695563
Iteration:      8,   Func. Count:     48,   Neg. LLF: 840.3430238214773
Iteration:      9,   Func. Count:     52,   Neg. LLF: 840.3430238214983
Optimization terminated successfully    (Exit mode 0)
            Current function value: 840.3430238214773
            Iterations: 9
            Function evaluations: 52
            Gradient evaluations: 9
Iteration:      1,   Func. Count:      6,   Neg. LLF: 876.7003857081925
Iteration:      2,   Func. Count:     13,   Neg. LLF: 889.3557645435415


19/06/2023

In [13]:
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss, df

### IN SAMPLE

In [91]:
def in_sample_forecast(data, lagged_covs, lagged_tickers, tickers, alpha, t_start, t_end):
    losses = {}

    for ticker in tickers:
        lagged_tickers_ticker = [tick for tick in lagged_tickers if tick.startswith(f'{ticker}_lag')]

        # Prepare the data
        X = data.loc[t_start:t_end, lagged_covs + lagged_tickers_ticker]
        y = data.loc[t_start:t_end, ticker]

        garch_model = arch.arch_model(y, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        # Perform in-sample forecast
        conditional_volatility = garch_results.conditional_volatility

        standardized_residuals = garch_results.resid / conditional_volatility
        z_value = np.percentile(standardized_residuals, alpha * 100)
        predicted_var = conditional_volatility * z_value

        loss, df = tick_loss(alpha, y, predicted_var)

        losses[ticker] = (loss, df)

    return losses

In [92]:
t_start = 255  # Start index of the test set
t_end = 279  # End index of the test set

losses_one_step_ahead_insample = in_sample_forecast(data, lagged_covs, lagged_tickers, tickers, alpha=0.05, t_start=t_start, t_end=t_end)

# Print losses for the test set observations
for ticker, loss_tuple in losses_one_step_ahead_insample.items():
    print(f"Ticker: {ticker}")
    loss, df = loss_tuple
    print(f"Tick Loss: {loss}")
    print(df)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 46.67421103301048
Iteration:      2,   Func. Count:     12,   Neg. LLF: 44.36956374278404
Iteration:      3,   Func. Count:     17,   Neg. LLF: 44.36932770269599
Iteration:      4,   Func. Count:     22,   Neg. LLF: 44.36893010880799
Iteration:      5,   Func. Count:     27,   Neg. LLF: 44.3688678173408
Iteration:      6,   Func. Count:     32,   Neg. LLF: 44.368688106275066
Iteration:      7,   Func. Count:     37,   Neg. LLF: 44.36831786910732
Iteration:      8,   Func. Count:     42,   Neg. LLF: 44.36723144960223
Iteration:      9,   Func. Count:     47,   Neg. LLF: 44.36436343283679
Iteration:     10,   Func. Count:     52,   Neg. LLF: 44.355779713391634
Iteration:     11,   Func. Count:     57,   Neg. LLF: 44.342581700974094
Iteration:     12,   Func. Count:     62,   Neg. LLF: 44.29956136943187
Iteration:     13,   Func. Count:     67,   Neg. LLF: 44.25567983227127
Iteration:     14,   Func. Count:     72,   Neg. LLF: 44.22344

### OUT OF SAMPLE

In [45]:
import arch

def one_step_ahead_VaR(data, lagged_covs, lagged_tickers, tickers, alpha, t):
    losses = {}

    for ticker in tickers:
        lagged_tickers_ticker = [tick for tick in lagged_tickers if tick.startswith(f'{ticker}_lag')]

        # Prepare the data
        train_data = data.loc[:t]
        test_data = data.loc[t:]

        X_train = train_data[lagged_covs + lagged_tickers_ticker]
        y_train = train_data[ticker]

        garch_model = arch.arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        X_test = test_data[lagged_covs + lagged_tickers_ticker]
        y_test = test_data[ticker]

        # Perform rolling forecast
        forecasts = garch_results.forecast(start=t, horizon=len(y_test))
        cond_vol_forecast = forecasts.variance[-1:].apply(lambda x: np.sqrt(x))
        mean_forecast = forecasts.mean[-1:]

        epsilon = np.random.normal(0, 1, size=(len(y_test), 10000))
        VaR_forecast = mean_forecast.values[0] + cond_vol_forecast.values[0] * np.percentile(epsilon, alpha * 100, axis=1)

        loss, df = tick_loss(alpha, y_test, VaR_forecast.flatten())

        losses[ticker] = (loss, df)

    return losses


In [46]:
losses_one_step_ahead = one_step_ahead_VaR(data, lagged_covs, lagged_tickers, tickers, alpha=0.05,t=255)

# Print losses
for ticker, loss_tuple in losses_one_step_ahead.items():
    print(f"Ticker: {ticker}")
    loss, df = loss_tuple
    print(f"Tick Loss: {loss}")
    print(df)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1159.7636996500732
Iteration:      2,   Func. Count:     13,   Neg. LLF: 849.6317953982542
Iteration:      3,   Func. Count:     20,   Neg. LLF: 937.2452486616035
Iteration:      4,   Func. Count:     28,   Neg. LLF: 840.3751966360928
Iteration:      5,   Func. Count:     33,   Neg. LLF: 840.3670078951413
Iteration:      6,   Func. Count:     38,   Neg. LLF: 840.3446163506405
Iteration:      7,   Func. Count:     43,   Neg. LLF: 840.3430701695563
Iteration:      8,   Func. Count:     48,   Neg. LLF: 840.3430238214773
Iteration:      9,   Func. Count:     52,   Neg. LLF: 840.3430238214983
Optimization terminated successfully    (Exit mode 0)
            Current function value: 840.3430238214773
            Iterations: 9
            Function evaluations: 52
            Gradient evaluations: 9
Iteration:      1,   Func. Count:      6,   Neg. LLF: 876.7003857081925
Iteration:      2,   Func. Count:     13,   Neg. LLF: 889.3557645435415


/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1932.3735579222598
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1197.0781359411403
Iteration:      3,   Func. Count:     20,   Neg. LLF: 857.6135671198253
Iteration:      4,   Func. Count:     26,   Neg. LLF: 851.4068874978846
Iteration:      5,   Func. Count:     31,   Neg. LLF: 851.398134712121
Iteration:      6,   Func. Count:     36,   Neg. LLF: 851.3546243312614
Iteration:      7,   Func. Count:     41,   Neg. LLF: 851.2924664505352
Iteration:      8,   Func. Count:     46,   Neg. LLF: 851.2780141089689
Iteration:      9,   Func. Count:     51,   Neg. LLF: 851.2757031171037
Iteration:     10,   Func. Count:     56,   Neg. LLF: 851.2749176515492
Iteration:     11,   Func. Count:     61,   Neg. LLF: 851.2748987022435
Iteration:     12,   Func. Count:     66,   Neg. LLF: 851.2748886874972
Iteration:     13,   Func. Count:     71,   Neg. LLF: 851.2748880716983
Optimization terminated successfully    (Exit mode 0)
         

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

# 3 months ahead

### IN SAMPLE

In [17]:
import pandas as pd
import numpy as np
import arch

def in_sample_forecast(data, lagged_covs, lagged_tickers, tickers, alpha):
    losses = {}

    for ticker in tickers:
        lagged_tickers_ticker = [tick for tick in lagged_tickers if tick.startswith(f'{ticker}_lag')]

        # Prepare the data
        X = data[lagged_covs + lagged_tickers_ticker]
        y = data[ticker]

        garch_model = arch.arch_model(y, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        # Perform in-sample forecast
        forecast_horizon = 3  # 3-month forecast horizon
        num_observations = len(y) - forecast_horizon

        for i in range(num_observations):
            start_index = i
            end_index = i + forecast_horizon
            X_test = X.iloc[start_index:end_index]
            y_test = y.iloc[start_index:end_index]

            forecasts = garch_results.forecast(start=start_index, horizon=forecast_horizon)
            cond_vol_forecast = forecasts.variance.iloc[-1].apply(lambda x: np.sqrt(x))
            mean_forecast = forecasts.mean.iloc[-1]

            epsilon = np.random.normal(0, 1, size=(forecast_horizon, 10000))
            VaR_forecast = mean_forecast + cond_vol_forecast * np.percentile(epsilon, alpha * 100, axis=1)

            loss, _ = tick_loss(alpha, y_test, VaR_forecast.flatten())
            losses[(ticker, start_index)] = loss

    return losses

In [18]:
alpha = 0.05

losses_insample_3months = in_sample_forecast(data, lagged_covs, lagged_tickers, tickers, alpha)

# Print losses for the in-sample forecast
for (ticker, start_index), loss in losses_insample_3months.items():
    print(f"Ticker: {ticker}, Start Index: {start_index}")
    print(f"Tick Loss: {loss}")


Iteration:      1,   Func. Count:      6,   Neg. LLF: 1220.2413727683897
Iteration:      2,   Func. Count:     13,   Neg. LLF: 891.4787040618978
Iteration:      3,   Func. Count:     20,   Neg. LLF: 965.078161133711
Iteration:      4,   Func. Count:     28,   Neg. LLF: 880.4715932694294
Iteration:      5,   Func. Count:     33,   Neg. LLF: 880.469766678912
Iteration:      6,   Func. Count:     38,   Neg. LLF: 880.4677341105024
Iteration:      7,   Func. Count:     43,   Neg. LLF: 880.4658609203732
Iteration:      8,   Func. Count:     48,   Neg. LLF: 880.4653909287848
Iteration:      9,   Func. Count:     53,   Neg. LLF: 880.4653460425961
Iteration:     10,   Func. Count:     58,   Neg. LLF: 880.4653451269519
Optimization terminated successfully    (Exit mode 0)
            Current function value: 880.4653451269519
            Iterations: 10
            Function evaluations: 58
            Gradient evaluations: 10


/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


AttributeError: ignored

### OUT OF SAMPLE

In [47]:
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss, df

In [68]:
import arch
import numpy as np

def three_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha, t):
    losses = {}

    for ticker in tickers:
        lagged_covs_t3 = [cov for cov in lagged_covs if int(cov.split('_lag')[1]) >= 3]
        lagged_tickers_t3_ticker = [tick for tick in lagged_tickers if tick.startswith(f'{ticker}_lag') and int(tick.split('_lag')[1]) >= 3]

        # Prepare the data
        train_data = data.loc[:t]
        test_data = data.loc[t:]

        X_train = train_data[lagged_covs_t3 + lagged_tickers_t3_ticker]
        y_train = train_data[ticker]

        garch_model = arch.arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        X_test = test_data[lagged_covs_t3 + lagged_tickers_t3_ticker]
        y_test = test_data[ticker]

        # Perform rolling forecast
        forecasts = garch_results.forecast(start=t, horizon=len(y_test))
        cond_vol_forecast = forecasts.variance[-1:].apply(lambda x: np.sqrt(x))
        mean_forecast = forecasts.mean[-1:]

        epsilon = np.random.normal(0, 1, size=(len(y_test), 10000))
        VaR_forecast = mean_forecast.values[0] + cond_vol_forecast.values[0] * np.percentile(epsilon, alpha * 100, axis=1)

        loss, df = tick_loss(alpha, y_test, VaR_forecast.flatten())

        losses[ticker] = (loss, df)

    return losses


In [69]:
losses_three_step_ahead = three_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha=0.05, t=255)

# Print losses
for ticker, loss_tuple in losses_three_step_ahead.items():
    print(f"Ticker: {ticker}")
    loss, df = loss_tuple
    print(f"Tick Loss: {loss}")
    print(df)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1077.6099789123105
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1213.622379783466
Iteration:      3,   Func. Count:     21,   Neg. LLF: 872.062964806496
Iteration:      4,   Func. Count:     28,   Neg. LLF: 770.4995249034941
Iteration:      5,   Func. Count:     33,   Neg. LLF: 770.4522576366847
Iteration:      6,   Func. Count:     38,   Neg. LLF: 770.4436016261117
Iteration:      7,   Func. Count:     43,   Neg. LLF: 770.4431022197208
Iteration:      8,   Func. Count:     48,   Neg. LLF: 770.4430590287492
Iteration:      9,   Func. Count:     53,   Neg. LLF: 770.443058525653
Optimization terminated successfully    (Exit mode 0)
            Current function value: 770.443058525653
            Iterations: 9
            Function evaluations: 53
            Gradient evaluations: 9
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1159.7636996500732
Iteration:      2,   Func. Count:     13,   Neg. LLF: 849.6317953982542
It

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 970.8821845361399
Iteration:      2,   Func. Count:     13,   Neg. LLF: 11769.889291743795
Iteration:      3,   Func. Count:     21,   Neg. LLF: 852.1121816370958
Iteration:      4,   Func. Count:     27,   Neg. LLF: 842.1713619359076
Iteration:      5,   Func. Count:     33,   Neg. LLF: 841.8299414570446
Iteration:      6,   Func. Count:     39,   Neg. LLF: 841.8199907902888
Iteration:      7,   Func. Count:     44,   Neg. LLF: 841.8199045567492
Iteration:      8,   Func. Count:     49,   Neg. LLF: 841.8199040606873
Optimization terminated successfully    (Exit mode 0)
            Current function value: 841.8199040606873
            Iterations: 8
            Function evaluations: 49
            Gradient evaluations: 8
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1932.3735579222598
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1197.0781359411403
Iteration:      3,   Func. Count:     20,   Neg. LLF: 857.613567119825

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

## Rolling forecast for evaluation:

In [51]:
import arch
import pandas as pd

def garch_rolling_forecast(data, ticker, alpha, dates):
    t_loss = 0
    dta = data.copy()

    for d in dates:
        split_date = d
        train_df = dta[dta.Date <= split_date]
        test_df = dta[dta.Date > split_date]
        test_df = test_df.reset_index(drop=True)

        garch_model = arch.arch_model(train_df[ticker], vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        forecasts = garch_results.forecast(start=0, horizon=3)
        cond_vol_forecast = forecasts.variance[-1:].apply(lambda x: pd.Series([garch_results.conditional_volatility.iloc[-1] * np.sqrt(x)]*3))
        mean_forecast = forecasts.mean[-1:]

        epsilon = np.random.normal(0, 1, size=(3, 10000))
        VaR_forecast = mean_forecast.values[0] + cond_vol_forecast.values[0] * np.percentile(epsilon, alpha * 100, axis=1)

        r = test_df[ticker][0:3]
        tickloss = tick_loss(alpha, r, VaR_forecast.flatten())

        t_loss += tickloss

    t_loss = t_loss / len(dates)
    return t_loss


In [52]:
dates3m = ['2022-05-01', '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01']

In [55]:
tickers = ['NVS', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'PFE', 'ROG']

for t in tickers:
    loss = garch_rolling_forecast(data, t, 0.05, dates3m)
    print(f'{t}: {loss}')

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1077.6099789123105
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1213.622379783466
Iteration:      3,   Func. Count:     21,   Neg. LLF: 872.062964806496
Iteration:      4,   Func. Count:     28,   Neg. LLF: 770.4995249034941
Iteration:      5,   Func. Count:     33,   Neg. LLF: 770.4522576366847
Iteration:      6,   Func. Count:     38,   Neg. LLF: 770.4436016261117
Iteration:      7,   Func. Count:     43,   Neg. LLF: 770.4431022197208
Iteration:      8,   Func. Count:     48,   Neg. LLF: 770.4430590287492
Iteration:      9,   Func. Count:     53,   Neg. LLF: 770.443058525653
Optimization terminated successfully    (Exit mode 0)
            Current function value: 770.443058525653
            Iterations: 9
            Function evaluations: 53
            Gradient evaluations: 9


/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


ValueError: ignored

# 6 months ahead

In [70]:
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss, df

In [71]:
import arch
import numpy as np

def six_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha, t):
    losses = {}

    for ticker in tickers:
        lagged_covs_t6 = [cov for cov in lagged_covs if int(cov.split('_lag')[1]) >= 6]
        lagged_tickers_t6_ticker = [tick for tick in lagged_tickers if tick.startswith(f'{ticker}_lag') and int(tick.split('_lag')[1]) >= 6]

        # Prepare the data
        train_data = data.loc[:t]
        test_data = data.loc[t:]

        X_train = train_data[lagged_covs_t6 + lagged_tickers_t6_ticker]
        y_train = train_data[ticker]

        garch_model = arch.arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        X_test = test_data[lagged_covs_t6 + lagged_tickers_t6_ticker]
        y_test = test_data[ticker]

        # Perform rolling forecast
        forecasts = garch_results.forecast(start=t, horizon=len(y_test) + 6)
        cond_vol_forecast = forecasts.variance[-1:].apply(lambda x: np.sqrt(x))
        mean_forecast = forecasts.mean[-1:]

        epsilon = np.random.normal(0, 1, size=(len(y_test) + 6, 10000))
        VaR_forecast = mean_forecast.values[0] + cond_vol_forecast.values[0] * np.percentile(epsilon, alpha * 100, axis=1)

        loss, df = tick_loss(alpha, y_test, VaR_forecast[-len(y_test):].flatten())

        losses[ticker] = (loss, df)

    return losses


In [72]:
losses_six_step_ahead = six_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha=0.05, t=255)

# Print losses
for ticker, loss_tuple in losses_six_step_ahead.items():
    print(f"Ticker: {ticker}")
    loss, df = loss_tuple
    print(f"Tick Loss: {loss}")
    print(df)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1077.6099789123105
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1213.622379783466
Iteration:      3,   Func. Count:     21,   Neg. LLF: 872.062964806496
Iteration:      4,   Func. Count:     28,   Neg. LLF: 770.4995249034941
Iteration:      5,   Func. Count:     33,   Neg. LLF: 770.4522576366847
Iteration:      6,   Func. Count:     38,   Neg. LLF: 770.4436016261117
Iteration:      7,   Func. Count:     43,   Neg. LLF: 770.4431022197208
Iteration:      8,   Func. Count:     48,   Neg. LLF: 770.4430590287492
Iteration:      9,   Func. Count:     53,   Neg. LLF: 770.443058525653
Optimization terminated successfully    (Exit mode 0)
            Current function value: 770.443058525653
            Iterations: 9
            Function evaluations: 53
            Gradient evaluations: 9
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1159.7636996500732
Iteration:      2,   Func. Count:     13,   Neg. LLF: 849.6317953982542
It

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      8,   Func. Count:     48,   Neg. LLF: 833.5212835725034
Iteration:      9,   Func. Count:     53,   Neg. LLF: 833.5212826052682
Optimization terminated successfully    (Exit mode 0)
            Current function value: 833.5212826052682
            Iterations: 9
            Function evaluations: 53
            Gradient evaluations: 9
Iteration:      1,   Func. Count:      6,   Neg. LLF: 970.8821845361399
Iteration:      2,   Func. Count:     13,   Neg. LLF: 11769.889291743795
Iteration:      3,   Func. Count:     21,   Neg. LLF: 852.1121816370958
Iteration:      4,   Func. Count:     27,   Neg. LLF: 842.1713619359076
Iteration:      5,   Func. Count:     33,   Neg. LLF: 841.8299414570446
Iteration:      6,   Func. Count:     39,   Neg. LLF: 841.8199907902888
Iteration:      7,   Func. Count:     44,   Neg. LLF: 841.8199045567492
Iteration:      8,   Func. Count:     49,   Neg. LLF: 841.8199040606873
Optimization terminated successfully    (Exit mode 0)
            Curren

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:     13,   Func. Count:     71,   Neg. LLF: 851.2748880716983
Optimization terminated successfully    (Exit mode 0)
            Current function value: 851.2748880716983
            Iterations: 13
            Function evaluations: 71
            Gradient evaluations: 13
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1038.1194476046944
Iteration:      2,   Func. Count:     13,   Neg. LLF: 826.6747012172495
Iteration:      3,   Func. Count:     20,   Neg. LLF: 1067.7587851808598
Iteration:      4,   Func. Count:     28,   Neg. LLF: 812.0828012904778
Iteration:      5,   Func. Count:     33,   Neg. LLF: 811.9997105039965
Iteration:      6,   Func. Count:     38,   Neg. LLF: 813.3829188680027
Iteration:      7,   Func. Count:     44,   Neg. LLF: 811.9764948566119
Iteration:      8,   Func. Count:     49,   Neg. LLF: 811.9674156140086
Iteration:      9,   Func. Count:     54,   Neg. LLF: 811.9667545266398
Iteration:     10,   Func. Count:     59,   Neg. LLF: 811.96672906996

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      1,   Func. Count:      6,   Neg. LLF: 1209.1403758137285
Iteration:      2,   Func. Count:     14,   Neg. LLF: 986.7696941746776
Iteration:      3,   Func. Count:     20,   Neg. LLF: 984.8933363378974
Iteration:      4,   Func. Count:     26,   Neg. LLF: 984.8920062555766
Iteration:      5,   Func. Count:     31,   Neg. LLF: 984.8919876942392
Iteration:      6,   Func. Count:     36,   Neg. LLF: 984.8919726042732
Iteration:      7,   Func. Count:     41,   Neg. LLF: 984.8919010711488
Iteration:      8,   Func. Count:     46,   Neg. LLF: 984.8917390018631
Iteration:      9,   Func. Count:     51,   Neg. LLF: 984.8913204943328
Iteration:     10,   Func. Count:     56,   Neg. LLF: 984.8902787142376
Iteration:     11,   Func. Count:     61,   Neg. LLF: 984.8878495712363
Iteration:     12,   Func. Count:     66,   Neg. LLF: 984.8834254476269
Iteration:     13,   Func. Count:     71,   Neg. LLF: 984.8782829269476
Iteration:     14,   Func. Count:     76,   Neg. LLF: 984.87628

## Rolling forecast for evaluation

In [81]:
import arch
import numpy as np

def rolling_window_evaluation_GARCH(data, tickers, lagged_covs, lagged_tickers, alpha, t, window_size, horizon):
    losses = {}

    for ticker in tickers:
        lagged_covs_t = [cov for cov in lagged_covs if int(cov.split('_lag')[1]) >= window_size]
        lagged_tickers_t_ticker = [tick for tick in lagged_tickers if tick.startswith(f'{ticker}_lag') and int(tick.split('_lag')[1]) >= window_size]

        for i in range(t, len(data) - window_size - horizon + 1):
            # Prepare the data
            train_data = data.loc[i: i + window_size - 1]
            test_data = data.loc[i + window_size - 1 : i + window_size - 1 + horizon - 1]

            X_train = train_data[lagged_covs_t + lagged_tickers_t_ticker]
            y_train = train_data[ticker]

            garch_model = arch.arch_model(y_train, vol='Garch', p=1, q=1)
            garch_results = garch_model.fit()

            X_test = test_data[lagged_covs_t + lagged_tickers_t_ticker]
            y_test = test_data[ticker]

            # Perform forecast
            forecasts = garch_results.forecast(start=i + window_size - 1, horizon=horizon)
            cond_vol_forecast = forecasts.variance[-1:].apply(lambda x: np.sqrt(x))
            mean_forecast = forecasts.mean[-1:]

            epsilon = np.random.normal(0, 1, size=(horizon, 10000))
            VaR_forecast = mean_forecast.values[0] + cond_vol_forecast.values[0] * np.percentile(epsilon, alpha * 100, axis=1)

            loss, df = tick_loss(alpha, y_test, VaR_forecast.flatten())

            if ticker not in losses:
                losses[ticker] = []

            losses[ticker].append((loss, df))

    return losses


In [82]:
window_size = 6
horizon = 6

In [83]:
# Call the rolling_window_evaluation_GARCH function
losses = rolling_window_evaluation_GARCH(data, tickers, lagged_covs, lagged_tickers, alpha, t, window_size, horizon)

# Print the losses for each ticker
for ticker, ticker_losses in losses.items():
    print(f'{ticker}:')
    for i, (loss, df) in enumerate(ticker_losses):
        print(f'Rolling Window {i+1} - Loss: {loss}, Degrees of Freedom: {df}')
    print()

TypeError: ignored

# 9 months ahead

In [73]:
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss, df

In [74]:
import arch
import numpy as np

def nine_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha, t):
    losses = {}

    for ticker in tickers:
        lagged_covs_t9 = [cov for cov in lagged_covs if int(cov.split('_lag')[1]) >= 9]
        lagged_tickers_t9_ticker = [tick for tick in lagged_tickers if tick.startswith(f'{ticker}_lag') and int(tick.split('_lag')[1]) >= 9]

        # Prepare the data
        train_data = data.loc[:t]
        test_data = data.loc[t:]

        X_train = train_data[lagged_covs_t9 + lagged_tickers_t9_ticker]
        y_train = train_data[ticker]

        garch_model = arch.arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        X_test = test_data[lagged_covs_t9 + lagged_tickers_t9_ticker]
        y_test = test_data[ticker]

        # Perform rolling forecast
        forecasts = garch_results.forecast(start=t, horizon=len(y_test) + 9)
        cond_vol_forecast = forecasts.variance[-1:].apply(lambda x: np.sqrt(x))
        mean_forecast = forecasts.mean[-1:]

        epsilon = np.random.normal(0, 1, size=(len(y_test) + 9, 10000))
        VaR_forecast = mean_forecast.values[0] + cond_vol_forecast.values[0] * np.percentile(epsilon, alpha * 100, axis=1)

        loss, df = tick_loss(alpha, y_test, VaR_forecast[-len(y_test):].flatten())

        losses[ticker] = (loss, df)

    return losses

In [75]:
losses_nine_step_ahead = nine_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha=0.05, t=255)

# Print losses
for ticker, loss_tuple in losses_nine_step_ahead.items():
    print(f"Ticker: {ticker}")
    loss, df = loss_tuple
    print(f"Tick Loss: {loss}")
    print(df)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1077.6099789123105
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1213.622379783466
Iteration:      3,   Func. Count:     21,   Neg. LLF: 872.062964806496
Iteration:      4,   Func. Count:     28,   Neg. LLF: 770.4995249034941
Iteration:      5,   Func. Count:     33,   Neg. LLF: 770.4522576366847
Iteration:      6,   Func. Count:     38,   Neg. LLF: 770.4436016261117
Iteration:      7,   Func. Count:     43,   Neg. LLF: 770.4431022197208
Iteration:      8,   Func. Count:     48,   Neg. LLF: 770.4430590287492
Iteration:      9,   Func. Count:     53,   Neg. LLF: 770.443058525653
Optimization terminated successfully    (Exit mode 0)
            Current function value: 770.443058525653
            Iterations: 9
            Function evaluations: 53
            Gradient evaluations: 9
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1159.7636996500732
Iteration:      2,   Func. Count:     13,   Neg. LLF: 849.6317953982542
It

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 970.8821845361399
Iteration:      2,   Func. Count:     13,   Neg. LLF: 11769.889291743795
Iteration:      3,   Func. Count:     21,   Neg. LLF: 852.1121816370958
Iteration:      4,   Func. Count:     27,   Neg. LLF: 842.1713619359076
Iteration:      5,   Func. Count:     33,   Neg. LLF: 841.8299414570446
Iteration:      6,   Func. Count:     39,   Neg. LLF: 841.8199907902888
Iteration:      7,   Func. Count:     44,   Neg. LLF: 841.8199045567492
Iteration:      8,   Func. Count:     49,   Neg. LLF: 841.8199040606873
Optimization terminated successfully    (Exit mode 0)
            Current function value: 841.8199040606873
            Iterations: 8
            Function evaluations: 49
            Gradient evaluations: 8
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1932.3735579222598
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1197.0781359411403
Iteration:      3,   Func. Count:     20,   Neg. LLF: 857.613567119825

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

# 12 months ahead

In [76]:
def tick_loss(alpha, returns, var):
    df = pd.DataFrame({'Return': returns, 'VaR': var})
    df['Indicator'] = np.where(df['Return'] < df['VaR'], 1, 0)

    t_loss = 0

    for i in df.index:
        t_loss += (
            alpha * (df['Return'][i] - df['VaR'][i]) * (1 - df['Indicator'][i])
            + (1 - alpha) * (df['VaR'][i] - df['Return'][i]) * df['Indicator'][i]
        )

    return t_loss, df

In [77]:
import arch
import numpy as np

def twelve_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha, t):
    losses = {}

    for ticker in tickers:
        lagged_covs_t12 = [cov for cov in lagged_covs if int(cov.split('_lag')[1]) >= 12]
        lagged_tickers_t12_ticker = [tick for tick in lagged_tickers if tick.startswith(f'{ticker}_lag') and int(tick.split('_lag')[1]) >= 12]

        # Prepare the data
        train_data = data.loc[:t]
        test_data = data.loc[t:]

        X_train = train_data[lagged_covs_t12 + lagged_tickers_t12_ticker]
        y_train = train_data[ticker]

        garch_model = arch.arch_model(y_train, vol='Garch', p=1, q=1)
        garch_results = garch_model.fit()

        X_test = test_data[lagged_covs_t12 + lagged_tickers_t12_ticker]
        y_test = test_data[ticker]

        # Perform rolling forecast
        forecasts = garch_results.forecast(start=t, horizon=len(y_test) + 12)
        cond_vol_forecast = forecasts.variance[-1:].apply(lambda x: np.sqrt(x))
        mean_forecast = forecasts.mean[-1:]

        epsilon = np.random.normal(0, 1, size=(len(y_test) + 12, 10000))
        VaR_forecast = mean_forecast.values[0] + cond_vol_forecast.values[0] * np.percentile(epsilon, alpha * 100, axis=1)

        loss, df = tick_loss(alpha, y_test, VaR_forecast[-len(y_test):].flatten())

        losses[ticker] = (loss, df)

    return losses


In [79]:
losses_twelve_step_ahead = twelve_step_ahead_VaR(data, tickers, lagged_covs, lagged_tickers, alpha=0.05, t=255)

# Print losses
for ticker, loss_tuple in losses_twelve_step_ahead.items():
    print(f"Ticker: {ticker}")
    loss, df = loss_tuple
    print(f"Tick Loss: {loss}")
    print(df)

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1077.6099789123105
Iteration:      2,   Func. Count:     13,   Neg. LLF: 1213.622379783466
Iteration:      3,   Func. Count:     21,   Neg. LLF: 872.062964806496
Iteration:      4,   Func. Count:     28,   Neg. LLF: 770.4995249034941
Iteration:      5,   Func. Count:     33,   Neg. LLF: 770.4522576366847
Iteration:      6,   Func. Count:     38,   Neg. LLF: 770.4436016261117
Iteration:      7,   Func. Count:     43,   Neg. LLF: 770.4431022197208
Iteration:      8,   Func. Count:     48,   Neg. LLF: 770.4430590287492
Iteration:      9,   Func. Count:     53,   Neg. LLF: 770.443058525653
Optimization terminated successfully    (Exit mode 0)
            Current function value: 770.443058525653
            Iterations: 9
            Function evaluations: 53
            Gradient evaluations: 9
Iteration:      1,   Func. Count:      6,   Neg. LLF: 1159.7636996500732
Iteration:      2,   Func. Count:     13,   Neg. LLF: 849.6317953982542
It

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1212.5634737557841
Iteration:      2,   Func. Count:     14,   Neg. LLF: 897.079383289743
Iteration:      3,   Func. Count:     21,   Neg. LLF: 853.3850129207542
Iteration:      4,   Func. Count:     28,   Neg. LLF: 833.5812277154168
Iteration:      5,   Func. Count:     33,   Neg. LLF: 833.540824940505
Iteration:      6,   Func. Count:     38,   Neg. LLF: 833.5236201942864
Iteration:      7,   Func. Count:     43,   Neg. LLF: 833.5213529893995
Iteration:      8,   Func. Count:     48,   Neg. LLF: 833.5212835725034
Iteration:      9,   Func. Count:     53,   Neg. LLF: 833.5212826052682
Optimization terminated successfully    (Exit mode 0)
            Current function value: 833.5212826052682
            Iterations: 9
            Function evaluations: 53
            Gradient evaluations: 9
Iteration:      1,   Func. Count:      6,   Neg. LLF: 970.8821845361399
Iteration:      2,   Func. Count:     13,   Neg. LLF: 11769.889291743795
I

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alter

Iteration:      1,   Func. Count:      6,   Neg. LLF: 1209.1403758137285
Iteration:      2,   Func. Count:     14,   Neg. LLF: 986.7696941746776
Iteration:      3,   Func. Count:     20,   Neg. LLF: 984.8933363378974
Iteration:      4,   Func. Count:     26,   Neg. LLF: 984.8920062555766
Iteration:      5,   Func. Count:     31,   Neg. LLF: 984.8919876942392
Iteration:      6,   Func. Count:     36,   Neg. LLF: 984.8919726042732
Iteration:      7,   Func. Count:     41,   Neg. LLF: 984.8919010711488
Iteration:      8,   Func. Count:     46,   Neg. LLF: 984.8917390018631
Iteration:      9,   Func. Count:     51,   Neg. LLF: 984.8913204943328
Iteration:     10,   Func. Count:     56,   Neg. LLF: 984.8902787142376
Iteration:     11,   Func. Count:     61,   Neg. LLF: 984.8878495712363
Iteration:     12,   Func. Count:     66,   Neg. LLF: 984.8834254476269
Iteration:     13,   Func. Count:     71,   Neg. LLF: 984.8782829269476
Iteration:     14,   Func. Count:     76,   Neg. LLF: 984.87628

/usr/local/lib/python3.10/dist-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


# MODEL COMPARISON